# Construyendo con los modelos de la familia Meta

## Introducción

En esta lección veremos:

- Exploración de los dos principales modelos de la familia Meta: Llama 3.1 y Llama 3.2
- Comprender los casos de uso y escenarios para cada modelo
- Ejemplo de código para mostrar las características únicas de cada modelo

## La familia de modelos Meta

En esta lección, exploraremos 2 modelos de la familia Meta o "Llama Herd": Llama 3.1 y Llama 3.2

Estos modelos tienen diferentes variantes y están disponibles en el marketplace de modelos de Github. Aquí tienes más información sobre cómo usar Github Models para [prototipar con modelos de IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Variantes de modelos:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Nota: Llama 3 también está disponible en Github Models pero no se cubrirá en esta lección*


## Llama 3.1

Con 405 mil millones de parámetros, Llama 3.1 entra en la categoría de LLMs de código abierto.

Este modelo es una mejora respecto a la versión anterior, Llama 3, ya que ofrece:

- Ventana de contexto más grande: 128k tokens frente a 8k tokens
- Mayor cantidad máxima de tokens de salida: 4096 frente a 2048
- Mejor soporte multilingüe: gracias al aumento de tokens de entrenamiento

Estas mejoras permiten que Llama 3.1 maneje casos de uso más complejos al crear aplicaciones de GenAI, incluyendo:
- Llamadas nativas a funciones: la capacidad de invocar herramientas y funciones externas fuera del flujo de trabajo del LLM
- Mejor rendimiento en RAG: gracias a la ventana de contexto más amplia
- Generación de datos sintéticos: la capacidad de crear datos efectivos para tareas como el ajuste fino


### Llamadas Nativas a Funciones

Llama 3.1 ha sido ajustado para ser más efectivo al realizar llamadas a funciones o herramientas. Además, cuenta con dos herramientas integradas que el modelo puede identificar como necesarias según la solicitud del usuario. Estas herramientas son:

- **Brave Search** - Se puede usar para obtener información actualizada como el clima realizando una búsqueda en la web
- **Wolfram Alpha** - Se puede usar para cálculos matemáticos más complejos, por lo que no es necesario escribir tus propias funciones.

También puedes crear tus propias herramientas personalizadas que el LLM pueda utilizar.

En el siguiente ejemplo de código:

- Definimos las herramientas disponibles (brave_search, wolfram_alpha) en el prompt del sistema.
- Enviamos un prompt de usuario que pregunta por el clima en una ciudad específica.
- El LLM responderá con una llamada a la herramienta Brave Search que se verá así `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Nota: Este ejemplo solo realiza la llamada a la herramienta; si deseas obtener los resultados, necesitarás crear una cuenta gratuita en la página de la API de Brave y definir la función tú mismo*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

A pesar de ser un LLM, una de las limitaciones que tiene Llama 3.1 es la multimodalidad. Es decir, la capacidad de usar diferentes tipos de entrada como imágenes como indicaciones y proporcionar respuestas. Esta capacidad es una de las principales características de Llama 3.2. Estas características también incluyen:

- Multimodalidad: tiene la capacidad de evaluar tanto indicaciones de texto como de imagen
- Variaciones de tamaño pequeño a mediano (11B y 90B): esto ofrece opciones de implementación flexibles,
- Variaciones solo de texto (1B y 3B): esto permite que el modelo se implemente en dispositivos edge o móviles y ofrece baja latencia

El soporte multimodal representa un gran avance en el mundo de los modelos de código abierto. El siguiente ejemplo de código toma tanto una imagen como una indicación de texto para obtener un análisis de la imagen por parte de Llama 3.2 90B.

### Soporte multimodal con Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## El aprendizaje no termina aquí, continúa el viaje

Después de completar esta lección, visita nuestra [colección de aprendizaje sobre IA Generativa](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para seguir ampliando tus conocimientos sobre IA Generativa.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que surjan del uso de esta traducción.
